# Self Test 1

## Inference with Markov models

In [3]:
import numpy as np

P1 = np.array([
    [0.3, 0.2, 0.5],
    [0.5, 0.3, 0.2],
    [0.2, 0.5, 0.3]])

S = ['A', 'B', 'C']

In [14]:
pE1 = P1[1,2] * P1[0,1]
logE1 = np.log(P1[1,2]) + np.log(P1[0,1])

print(round(pE1,2))
print(round(logE1,2))

0.04
-3.22


In [15]:
pE2 = P1[0,2] * P1[1,0]
logE2 = np.log(P1[0,2]) + np.log(P1[1,0])

print(round(pE2,2))
print(round(logE2,2))

0.25
-1.39


## Sampling a Markov model

In [6]:
import numpy.random as rnd

states = ['A', 'B', 'C']
indices = range(len(states))
state2index = dict(zip(states, indices))
index2state = dict(zip(indices, states))

def generateStateSequence(X0, P, tau):
    sseq = [X0]
    iold = state2index[X0]
    for t in range(tau-1):
        inew = rnd.choice(indices, p=P[:,iold])
        sseq.append(index2state[inew])
        iold = inew
    return sseq

def generate_episoids(X0, P, tau=200, times=1000):
    episoids = np.zeros((times, tau)).astype(object)
    for i in range(times):
        sequence = generateStateSequence(X0, P, tau)
        episoids[i] = np.array(sequence)
    return episoids

In [17]:
episoids1 = generate_episoids('A', P1)

In [25]:
# print sequence nice
for i, state in enumerate(episoids1[0]):
    print(state, end=' ')

A A B B A A B A B B C A A C A A A C C B B A C C A B C A B A B A B C A B C A B A C A A A B C C B C A B C A A A A A B C C A B B B C A B A B C C C C A C C A B C A B C A B B C C A A B B C C B C C A B C B A B C A C C A B C C B C B B C A A B C B C B B A A B C A A B C A C C A B B C C A B B C A B B A C C C B C A C A B C A B C A C B A B A B B C B A B C C B C B C A A B B C A A A B C A B C C A C A A A B A B

In [8]:
def compute_avgloglike(episoids):
    log_sums = [] 
    for n in range(episoids1.shape[0]):
        # current state
        log_sum = 0   
        t = states.index(episoids1[n,0])
        for l in range(1,episoids1.shape[1]):
            # successive state
            t_1 = states.index(episoids1[n,l])
            # compute incremental log likelihood
            log_sum += np.log(P1[t_1,t])
            # increment the time stamp
            t = t_1
        # print(log_sum)
        log_sums.append(log_sum)

    avg = sum(log_sums) / len(log_sums)
    return avg

In [9]:
avgloglike = compute_avgloglike(episoids1)
print(round(avgloglike, 2))

-205.06


## Learning a Markov model

In [10]:
state_num = len(S)

def estimate_transisions(episoids):
    matP = np.zeros((state_num,state_num))
    
    for s_idx, state in enumerate(S):
        # find position of each state in each episode
        epps, states = np.where((episoids == state))

        for ep, s in zip(epps, states):
            # if last state of the sequence then skip
            if (s+1) >= episoids.shape[1]:
                continue

            # get successive state
            succ_state = episoids[ep, s+1]
            # get successive state idx
            ss_idx = S.index(succ_state)
            # increment successive state count
            matP[ss_idx,s_idx] += 1
    
    matP /= matP.sum(axis=1)
    return matP

In [11]:
Pmle = estimate_transisions(episoids1)

print(Pmle.round(2))
print()
print(P1)

[[0.3 0.2 0.5]
 [0.5 0.3 0.2]
 [0.2 0.5 0.3]]

[[0.3 0.2 0.5]
 [0.5 0.3 0.2]
 [0.2 0.5 0.3]]


## Stationary distribution a Markov model

In [12]:
import numpy as np
import numpy.linalg as la

m = P1.shape[0]
b = np.hstack((np.zeros(m), 1))
I = np.eye(m)
A = np.vstack((I-P1, np.ones(m)))
pi = la.lstsq(A, b)[0]

print(pi)

[0.33333333 0.33333333 0.33333333]
